In [1]:
import Evaluasi
import numpy as np
import matplotlib.pyplot as plt 
import h5py
import cv2
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model

loss = []
val_loss = []
with open('D:/Kuliah/Thesis/Pengujian/best/pengujian2/History_Train.pkl','rb') as f:
    log = pickle.load(f)
    for i in range(len(log)):
        loss.append(np.amin(log[i].get('loss')))
        val_loss.append(np.amin(log[i].get('val_loss')))

indeks = np.where(val_loss == np.amin(val_loss))[0]
predModel = load_model('D:/Kuliah/Thesis/Pengujian/best/pengujian2/Fold'+str(indeks[0]+1)+'.hdf5')

In [13]:
def preprocessing(ori):
    h,w,c = ori.shape
    crop_img = ori[120:120+h, 960:960+w]
    res = cv2.resize(crop_img,dsize=(320,320),interpolation=cv2.INTER_CUBIC)
    gray = cv2.cvtColor(res,cv2.COLOR_BGR2GRAY)
    return crop_img,gray

cap = cv2.VideoCapture('D:/Kuliah/Thesis/Data/cctv/20191209070000-20191209160000/IP Camera1_Nvr Penelitian_Nvr Penelitian_20191209080838_20191209080954_242964.mp4')
rasio = (960/320)

if (cap.isOpened()==False):
    print("Error")

while(cap.isOpened()):
    ret,frame = cap.read()
    if ret == True:
        show,prep = preprocessing(frame)
        heatmapPrediction = predModel.predict(np.expand_dims(prep,axis=[0,-1]))
        
        mean = np.mean([heatmapPrediction[0,:,:,i].max() for i in range(heatmapPrediction.shape[-1])])
        print(mean)
        
        keypointPred = Evaluasi.KeypointPrediction(heatmapPrediction[0])
        if len(keypointPred)>0:
            keypointPred = keypointPred*rasio
            # print(keypointPred)
            color = (0, 255, 0)
            cv2.imshow('Heatmap',np.sum(np.squeeze(heatmapPrediction,axis=0),axis=-1))

            if mean>0.3:
                for j in range(keypointPred.shape[0]):
                    cv2.polylines(show,[np.array([keypointPred[j,k,l,::-1]for l in range(keypointPred.shape[2])for k in range(keypointPred.shape[1])],np.int32)], True, color, 2)

                    # box = keypointPred[j,0,:,:]
                    # start = (int(box[0,1]),int(box[0,0]))
                    # end = (int(box[1,1]),int(box[1,0]))
                    # cv2.rectangle(show,start,end,color,3)

            cv2.imshow('Frame',show)

            key = cv2.waitKey(1)
            if key == ord('q'):
                break
            if key == ord('p'):
                cv2.waitKey(-1)
    else:
        break

cap.release()
cv2.destroyAllWindows()

0.00022848944
0.00034824252
0.0003673601
0.00033436116
0.00036009328
0.00035130797
0.00031294228
0.00034524518
0.00033870278
0.0003576551
0.00034552335
0.00035605935
0.00034689656
0.0003562546
0.00042164949
0.00042261888
0.0004136501
0.0003855804
0.000387501
0.00038662687
0.0003981997
0.00041920558
0.0004599982
0.00041498273
0.0004202369
0.00041375155
0.00045196823
0.0004757943
0.00044599865
0.0004502149
0.0004902191
0.00048599494
0.00048662425
0.0005052869
0.00054762245
0.0005428168
0.000549754
0.0005179175
0.0005266558
0.00064978923
0.0005301755
0.0005723345
0.00056367053
0.0005882386
0.0006069475
0.0005943604
0.000566793
0.00058677734
0.000582036
0.00057900697
0.00041853232
0.0002862835
0.00030533317
0.00031939714
0.00034090097
0.00031583314
0.000311538
0.0004063958
0.00046216213
0.00047527003
0.00047730035
0.0005328533
0.000541125
0.0005346647
0.0005923492
0.0006146747
0.00062815356
0.00052254833
0.00043889895
0.00040187588
0.00041142822
0.00045581837
0.00046307323
0.00043866894
0.